In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# import matplotlib.pyplot as plt
# from joblib import Parallel, delayed
# from sklearn.ensemble import RandomForestRegressor

In [ ]:
strs_ref = pd.read_csv('overall/spr/spr_ms_256.csv')
# strs_ref


In [30]:
x = pd.read_csv('Coord/mesh_256.csv')
x = x.loc[:, ~x.columns.str.contains('^Unnamed')]
print(x.head());


        X         Y
0  0.0000  0.000000
1  0.1875  0.171875
2  0.3750  0.343750
3  0.5625  0.515625
4  0.7500  0.687500


In [32]:
y = strs_ref
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]
y = y.loc[:, ~y.columns.str.contains('^x')]
y = y.loc[:, ~y.columns.str.contains('^y')]
y.head()


,sigma_x_spr,sigma_y_spr,sigma_xy_spr
0,0.021022,0.012042,0.017168
1,0.012886,0.010272,0.011452
2,0.009598,0.007714,0.008643
3,0.010346,0.008532,0.009383
4,0.010952,0.009053,0.009967


In [ ]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []

    nodes_increment = (last_layer_nodes - first_layer_nodes) / (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    return layers


In [ ]:
import time
start = time.time()


In [ ]:
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    #newmodel
    model1 = Sequential()
    n_nodes = FindLayerNodesLinear(
        n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i == 1:
            model1.add(Dense(first_layer_nodes, activation=activation_func))
        else:
            model1.add(Dense(n_nodes[i-1], activation=activation_func))

    model1.add(Dense(units=3, activation='linear'))
    model1.compile(
        loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
    return model1


seed = 7
np.random.seed(seed)


In [ ]:
X = x
Y = y


In [2]:

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [256, 512, 1024, 2048]
epochs = [50, 100, 150, 200, 300]
n_layers = [5, 8, 10]
first_layer_nodes = [50, 60, 80]
last_layer_nodes = [5, 10]
activation_funcs = ['relu', 'sigmoid', 'tanh']
param_grid = dict(epochs=epochs, batch_size=batch_size, n_layers=n_layers,
                  first_layer_nodes=first_layer_nodes, last_layer_nodes=last_layer_nodes, activation_func=activation_funcs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    n_jobs=-1, cv=5, verbose=2)
#print(1, neurons1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


NameError: name 'KerasRegressor' is not defined

In [ ]:
end = time.time()
print("The time of execution of above program is :", end-start)
